In [21]:
import pandas as pd
import numpy as np

file='joinedhpv.csv'
df=pd.read_csv(file)
print (df.columns)
cols=['enc_date','mrn','HPV']
df=df[cols]
print (df.head())
print (df.HPV.value_counts())

Index(['PID', 'p16', 'HPV', 'mrn', 'enc_date', 'proc_desc', 'blocks',
       'SCANNED', 'Name', 'protocol', 'PCR SUBTYPE',
       'HPV+ PCR:\n 0=NEG,\n 1=POS', 'accession #', 'Date collected',
       'p16 IHC \n0=neg, \n1=pos', 'in house p16\n0=no, \n1=yes',
       'other HPV/Validation?', 'p16 accuracy',
       'PCR Validation value (TP, TN, FP, FN)', 'DISCREPANT? \n0=NO, \n1=YES',
       'Primary code:\n1: Oropharynx\n2: Hypopharynx\n3: Larynx\n4: Oral Cavity\n5: Nasopharynx\n6: Unknown',
       'path - primary'],
      dtype='object')
     enc_date      mrn       HPV
0  2016-06-21  1000196       NaN
1  2013-03-01  1012616  negative
2  2011-12-12  1325320       NaN
3  2018-11-05  1365262  positive
4  2010-02-03  1365262  negative
positive     121
negative      94
manual         6
equivocal      1
Name: HPV, dtype: int64


In [22]:
def get_HPV_status(df):
    hasPos=False
    hasNeg=False
    for index,row in df.iterrows():
        status=row['HPV']
        if status=='positive':
            hasPos=True
        elif status=='negative':
            hasNeg=True
    if hasPos:
        return 1
    elif hasNeg:
        return 0
    else:
        return 2

df=df.sort_values('enc_date')
mrns=df.mrn.unique()
ptx_info=[]
for mrn in mrns:
    sub_df=df[df['mrn']==mrn]
    ptx_info.append(sub_df)
data=[]
for ptx in ptx_info:
    mrn=ptx.iloc[0]['mrn']
    status=get_HPV_status(ptx)
    data.append([mrn,status])
df_export=pd.DataFrame(data=data,columns=['mrn','HPV_status'])
print (df_export.head())
#df_export.to_csv('RM_HPV.csv',index=False)

       mrn  HPV_status
0  2952217           0
1  2952781           1
2  2954600           0
3  2957771           0
4  2989110           2


In [23]:
df=pd.read_csv('val_patients_no_recur.csv')
print (df.columns)
cols=['MRN','HPV']
df=df[cols]
df.MRN=df.MRN.str.replace('-','').astype(int)
df.HPV=df.HPV.str.replace(' ','')
print (df.HPV.value_counts())
map_={'0':0,'1':1,'HPV16':1,'HPV18':1,'HPV33':1,'HPV+':1,'HPV-':0}
df.HPV=df.HPV.map(map_)
print (df.HPV.value_counts())

df=df.dropna(subset=['HPV'])
df.columns=['mrn','HPV_status']

df_export=df_export.append(df)
print(df_export.shape[0])
df_export=df_export.drop_duplicates(subset='mrn',keep='last')
print(df_export.shape[0])
print (df_export)
df_export.to_csv('RM_HPV.csv',index=False)

Index(['Name', 'MRN', 'Gender', 'Race', 'DOB', 'Age', 'Arm', 'RECIST response',
       'Date of Study Entry', 'Last Treatment Date', 'Date of Progression',
       'Date of Death', 'trial', 'HPV', 'stage', 'T', 'N', 'M', 'where'],
      dtype='object')
HPV16        83
0            36
1            33
.            17
HPV33         4
HPV-          3
HPV+          1
HPV18         1
EQUIVOCAL     1
Name: HPV, dtype: int64
1.0    122
0.0     39
Name: HPV, dtype: int64
550
509
         mrn  HPV_status
0    2952217         0.0
1    2952781         1.0
2    2954600         0.0
4    2989110         2.0
5    1942810         2.0
..       ...         ...
240  3728585         1.0
241  3709726         1.0
242  3708841         1.0
243  3034289         1.0
245  3724219         1.0

[509 rows x 2 columns]
